In [4]:
import arcpy,os;

fgdb = os.getcwd() + os.sep + 'source.gdb';

grid = fgdb + os.sep + 'CRWU_CREAT_Grid_Projections';
grid_cnt = arcpy.GetCount_management(grid)[0];
stat = fgdb + os.sep + 'D4EMLite_PREC_Details';
stat_cnt = arcpy.GetCount_management(stat)[0];
coop = fgdb + os.sep + 'COOP_STATIONS_TO_USE';
coop_cnt = arcpy.GetCount_management(coop)[0];
isd  = fgdb + os.sep + 'ISD_STATIONS_TO_USE';
isd_cnt = arcpy.GetCount_management(isd)[0];

print("Grid Projections: " + str(grid_cnt));
print("Orig Stations: " + str(stat_cnt));
print("COOP Stations: " + str(coop_cnt));
print("ISD Stations: " + str(isd_cnt));


Grid Projections: 24743
Orig Stations: 8158
COOP Stations: 1851
ISD Stations: 3293


In [5]:
wrkfgdb = os.getcwd() + os.sep + 'working.gdb';

if not arcpy.Exists(wrkfgdb):
   arcpy.CreateFileGDB_management(
       os.path.dirname(wrkfgdb)
      ,os.path.basename(wrkfgdb)
   );

wrkstatnear = wrkfgdb + os.sep + 'D4EMLite_PREC_Details_near';

if arcpy.Exists(wrkstatnear):
    arcpy.Delete_management(wrkstatnear);

print("  copying original stations");
arcpy.management.Copy(
     in_data  = stat
    ,out_data = wrkstatnear
);

print("  adding indexes");
arcpy.management.AddIndex(
     in_table   = wrkstatnear
    ,fields     = 'StationId'
    ,index_name = 'StationId_IDX'
);
    
print("  calculating coop near values");
arcpy.analysis.Near(
     in_features   = wrkstatnear
    ,near_features = coop
    ,search_radius = '1 kilometers'
    ,location      = 'NO_LOCATION'
    ,method        = 'GEODESIC'
);

arcpy.management.AlterField(
     in_table        = wrkstatnear
    ,field           = 'NEAR_FID'
    ,new_field_name  = 'NEAR_FID_COOP'
    ,new_field_alias = 'NEAR_FID_COOP'
);

arcpy.management.AlterField(
     in_table        = wrkstatnear
    ,field           = 'NEAR_DIST'
    ,new_field_name  = 'NEAR_DIST_COOP'
    ,new_field_alias = 'NEAR_DIST_COOP'
);

print("  calculating isd near values");
arcpy.analysis.Near(
     in_features   = wrkstatnear
    ,near_features = isd
    ,search_radius = '1 kilometers'
    ,location      = 'NO_LOCATION'
    ,method        = 'GEODESIC'
);

arcpy.management.AlterField(
     in_table        = wrkstatnear
    ,field           = 'NEAR_FID'
    ,new_field_name  = 'NEAR_FID_ISD'
    ,new_field_alias = 'NEAR_FID_ISD'
);

arcpy.management.AlterField(
     in_table        = wrkstatnear
    ,field           = 'NEAR_DIST'
    ,new_field_name  = 'NEAR_DIST_ISD'
    ,new_field_alias = 'NEAR_DIST_ISD'
);

print("  joining to coop on 6-byte identifiers");
arcpy.management.JoinField(
     in_data         = wrkstatnear
    ,in_field        = 'StationId'
    ,join_table      = coop
    ,join_field      = 'station_id_six'
    ,fields          = ['station_id_six']
);

print("DONE");


  copying original stations
  adding indexes
  calculating coop near values
  calculating isd near values
  joining to coop on 6-byte identifiers
DONE


In [9]:
rezfgdb = os.getcwd() + os.sep + 'results.gdb';

if not arcpy.Exists(rezfgdb):
   arcpy.CreateFileGDB_management(
       os.path.dirname(rezfgdb)
      ,os.path.basename(rezfgdb)
   );

rezstat = rezfgdb + os.sep + 'D4EMLite_PREC_Details';

if arcpy.Exists(rezstat):
    arcpy.Delete_management(rezstat);
    
arcpy.CreateFeatureclass_management(
     out_path      = rezfgdb
    ,out_name      = 'D4EMLite_PREC_Details'
    ,geometry_type = "POINT"
    ,template      = stat
    ,has_m         = "DISABLED"
    ,has_z         = "DISABLED"
    ,spatial_reference = arcpy.SpatialReference(4326) 
);

fldin = [
     'StationId'
    ,'DataType'
    ,'FileName'
    ,'Id'
    ,'Lat'
    ,'Long'
    ,'Scenario'
    ,'Constituent'
    ,'SDate'
    ,'EDate'
    ,'YrCount'
    ,'Value'
    ,'StaNam'
    ,'NEAR_FID_COOP'
    ,'NEAR_FID_ISD'
    ,'station_id_six'
    ,'SHAPE@'
];

fldout = [
     'StationId'
    ,'DataType'
    ,'FileName'
    ,'Id'
    ,'Lat'
    ,'Long'
    ,'Scenario'
    ,'Constituent'
    ,'SDate'
    ,'EDate'
    ,'YrCount'
    ,'Value'
    ,'StaNam'
    ,'SHAPE@'
];

fldcoop = [
     'station_id'
    ,'station_name'
    ,'state'
    ,'start_date_clean'
    ,'end_date_clean'
    ,'latitude'
    ,'longitude'
    ,'in_basins'
    ,'break_with_basins'
    ,'network'
    ,'start_date_to_use_clean'
    ,'end_date_to_use_clean'
    ,'station_id_six'
    ,'year_count'
    ,'SHAPE@'
];

fldisd = [
     'station_id'
    ,'station_name'
    ,'state'
    ,'start_date'
    ,'end_date'
    ,'latitude'
    ,'longitude'
    ,'in_basins'
    ,'break_with_basins'
    ,'network'
    ,'year_count'
    ,'SHAPE@'
];

with arcpy.da.InsertCursor(rezstat,fldout) as outcur:

    with arcpy.da.SearchCursor(wrkstatnear,fldin) as incur:
        
        for row in incur:
            
            if row[13] == -1 and row[14] == -1 and row[15] is None:
                outcur.insertRow((
                     row[0]
                    ,row[1]
                    ,row[2]
                    ,row[3]
                    ,row[4]
                    ,row[5]
                    ,row[6]
                    ,row[7]
                    ,row[8]
                    ,row[9]
                    ,row[10]
                    ,row[11]
                    ,row[12]
                    ,row[16]
                ));
          
    with arcpy.da.SearchCursor(coop,fldcoop) as incur:
        
        for row in incur:
            
            outcur.insertRow((
                 row[12]
                ,'WdmFinal'
                ,row[2].lower() + str(row[12]) + '.txt'
                ,1
                ,row[5]
                ,row[6]
                ,'OBSERVED'
                ,'PREC'
                ,row[10]
                ,row[11]
                ,row[13]
                ,0.0
                ,row[1]
                ,row[14]
            ));
    z="""         
    with arcpy.da.SearchCursor(isd,fldisd) as incur:
        
        for row in incur:
            
            outcur.insertRow((
                 row[0]
                ,row[1]
                ,row[2]
                ,row[3]
                ,row[4]
                ,row[5]
                ,row[6]
                ,row[7]
                ,row[8]
                ,row[9]
                ,row[10]
                ,row[11]
                ,row[12]
                ,row[13]
            ));
    """;
        

C:\Users\PDziemiela\Documents\GitHub\Data-Processing-for-SWC-and-SWMM-CAT\working.gdb\D4EMLite_PREC_Details_near


In [10]:
stat_cnt = arcpy.GetCount_management(rezstat)[0];

print("New Stations: " + str(stat_cnt));


New Stations: 7845
